# 🥔 Phase 4: Data Augmentation & Dataset Splitting

This notebook augments and splits the preprocessed potato disease dataset for model training.

**Features:**
- Stratified split into train/validation/test sets (70/20/10)
- Realistic data augmentation for training set
- Maintains class balance across all subsets
- Preserves original images in validation/test sets
- Generates comprehensive split statistics

## 1. Mount Google Drive

In [ ]:
from google.colab import drive
import os
import shutil

MOUNT_PATH = '/content/drive'

def mount_drive():
    """Mount Google Drive with automatic error handling."""
    if os.path.exists(os.path.join(MOUNT_PATH, 'MyDrive')):
        print('✅ Google Drive is already mounted!')
        return True
    
    if os.path.exists(MOUNT_PATH):
        print('🔄 Clearing existing mount point...')
        try:
            drive.flush_and_unmount()
        except:
            pass
        if os.path.exists(MOUNT_PATH):
            try:
                shutil.rmtree(MOUNT_PATH)
            except:
                pass
    
    try:
        drive.mount(MOUNT_PATH)
        print('✅ Google Drive mounted successfully!')
        return True
    except Exception as e:
        print(f'❌ Mount failed: {e}')
        print('🔧 Try: Runtime → Restart runtime, then run again')
        return False

mount_drive()

## 2. Install Dependencies & Configuration

In [ ]:
# Install required packages
!pip install -q pillow opencv-python-headless albumentations tqdm scikit-learn

import os
import shutil
import json
import random
from pathlib import Path
from datetime import datetime
from collections import defaultdict
from tqdm import tqdm

import cv2
import numpy as np
from PIL import Image
import albumentations as A
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print("✅ Dependencies loaded!")
print(f"🎲 Random seed: {RANDOM_SEED}")
print(f"📦 Albumentations version: {A.__version__}")

In [ ]:
# ===== CONFIGURATION =====

# Source: Preprocessed dataset from Phase 3
SOURCE_DIR = "/content/drive/MyDrive/DrukFarm/data/preprocessed_potato_dataset"

# Output: Final split and augmented dataset
OUTPUT_DIR = "/content/drive/MyDrive/DrukFarm/data/final_potato_dataset"

# Split ratios
TRAIN_RATIO = 0.70  # 70% for training
VAL_RATIO = 0.20    # 20% for validation
TEST_RATIO = 0.10   # 10% for testing

# Augmentation settings
AUGMENT_TRAINING = True          # Apply augmentation to training set
AUGMENTATION_FACTOR = 3          # Number of augmented versions per original image
TARGET_SIZE = (224, 224)         # Image size (should match preprocessing)

# Valid extensions
VALID_EXTENSIONS = {'.jpg', '.jpeg', '.png'}

print("✅ Configuration loaded!")
print(f"\n📁 Source: {SOURCE_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"\n📊 Split ratios:")
print(f"   • Training: {TRAIN_RATIO*100:.0f}%")
print(f"   • Validation: {VAL_RATIO*100:.0f}%")
print(f"   • Testing: {TEST_RATIO*100:.0f}%")
print(f"\n🔄 Augmentation factor: {AUGMENTATION_FACTOR}x (training only)")

## 3. Define Augmentation Pipeline

In [ ]:
def create_augmentation_pipeline():
    """
    Create a robust augmentation pipeline using Albumentations.
    These transformations simulate real-world variations in leaf images.
    Compatible with albumentations v1.4+
    """
    transform = A.Compose([
        # Geometric transformations
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.Rotate(limit=30, p=0.5, border_mode=cv2.BORDER_REFLECT),
        A.Affine(
            scale=(0.9, 1.1),
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
            rotate=(-15, 15),
            shear=(-5, 5),
            p=0.5,
            mode=cv2.BORDER_REFLECT
        ),
        
        # Random crop and resize (compatible syntax)
        A.RandomScale(scale_limit=0.15, p=0.3),
        A.PadIfNeeded(
            min_height=TARGET_SIZE[0],
            min_width=TARGET_SIZE[1],
            border_mode=cv2.BORDER_REFLECT,
            p=1.0
        ),
        A.RandomCrop(
            height=TARGET_SIZE[0],
            width=TARGET_SIZE[1],
            p=0.5
        ),
        
        # Color/lighting transformations (preserve disease characteristics)
        A.OneOf([
            A.RandomBrightnessContrast(
                brightness_limit=0.2,
                contrast_limit=0.2,
                p=1
            ),
            A.ColorJitter(
                brightness=0.15,
                contrast=0.15,
                saturation=0.1,
                hue=0.05,
                p=1
            ),
        ], p=0.5),
        
        # Slight blur to simulate focus variations
        A.OneOf([
            A.GaussianBlur(blur_limit=(3, 5), p=1),
            A.MotionBlur(blur_limit=3, p=1),
        ], p=0.2),
        
        # Add slight noise (simulates camera sensor noise)
        A.GaussNoise(var_limit=(5, 20), p=0.2),
        
        # Ensure output size is correct
        A.Resize(height=TARGET_SIZE[0], width=TARGET_SIZE[1]),
    ])
    
    return transform


# Create pipeline
augmentation_pipeline = create_augmentation_pipeline()
print("✅ Augmentation pipeline created!")
print("\n📋 Augmentation techniques:")
print("   • Horizontal/Vertical flip")
print("   • Rotation (±30°)")
print("   • Affine transforms (scale, translate, shear)")
print("   • Random crop & scale")
print("   • Brightness/contrast adjustment")
print("   • Color jitter")
print("   • Gaussian blur/noise")

## 4. Scan Source Dataset

In [ ]:
def scan_dataset(source_dir: str) -> dict:
    """
    Scan source dataset and collect all image paths by class.
    """
    dataset = {}
    
    if not os.path.exists(source_dir):
        print(f"❌ Source directory not found: {source_dir}")
        return dataset
    
    class_dirs = [d for d in os.listdir(source_dir) 
                  if os.path.isdir(os.path.join(source_dir, d))]
    
    print(f"📁 Found {len(class_dirs)} classes\n")
    
    total_images = 0
    for class_name in sorted(class_dirs):
        class_path = os.path.join(source_dir, class_name)
        
        # Get all valid images
        images = [
            os.path.join(class_path, f)
            for f in os.listdir(class_path)
            if Path(f).suffix.lower() in VALID_EXTENSIONS
        ]
        
        dataset[class_name] = images
        total_images += len(images)
        print(f"   🏷️ {class_name}: {len(images)} images")
    
    print(f"\n📊 Total images: {total_images}")
    return dataset


# Scan dataset
print("🔍 Scanning source dataset...\n")
source_dataset = scan_dataset(SOURCE_DIR)

## 5. Stratified Dataset Split

In [ ]:
def stratified_split(dataset: dict, train_ratio: float, val_ratio: float, 
                     test_ratio: float, seed: int = 42) -> dict:
    """
    Perform stratified split maintaining class balance.
    Returns dict with train/val/test splits per class.
    """
    splits = {
        'train': {},
        'validation': {},
        'test': {}
    }
    
    print("📊 Performing stratified split...\n")
    
    for class_name, images in dataset.items():
        # Shuffle images
        shuffled = images.copy()
        random.Random(seed).shuffle(shuffled)
        
        n_total = len(shuffled)
        n_test = max(1, int(n_total * test_ratio))
        n_val = max(1, int(n_total * val_ratio))
        n_train = n_total - n_val - n_test
        
        # Split
        splits['test'][class_name] = shuffled[:n_test]
        splits['validation'][class_name] = shuffled[n_test:n_test + n_val]
        splits['train'][class_name] = shuffled[n_test + n_val:]
        
        print(f"   🏷️ {class_name}:")
        print(f"      Train: {len(splits['train'][class_name])} | "
              f"Val: {len(splits['validation'][class_name])} | "
              f"Test: {len(splits['test'][class_name])}")
    
    # Summary
    total_train = sum(len(v) for v in splits['train'].values())
    total_val = sum(len(v) for v in splits['validation'].values())
    total_test = sum(len(v) for v in splits['test'].values())
    total = total_train + total_val + total_test
    
    print(f"\n📊 SPLIT SUMMARY:")
    print(f"   • Training: {total_train} images ({total_train/total*100:.1f}%)")
    print(f"   • Validation: {total_val} images ({total_val/total*100:.1f}%)")
    print(f"   • Testing: {total_test} images ({total_test/total*100:.1f}%)")
    
    return splits


# Perform split
dataset_splits = stratified_split(
    source_dataset, 
    TRAIN_RATIO, 
    VAL_RATIO, 
    TEST_RATIO,
    seed=RANDOM_SEED
)

## 6. Create Output Directory Structure

In [ ]:
def create_output_structure(output_dir: str, classes: list) -> None:
    """
    Create the output directory structure.
    """
    # Remove existing output directory if exists
    if os.path.exists(output_dir):
        print(f"⚠️ Removing existing output directory...")
        shutil.rmtree(output_dir)
    
    # Create structure
    subsets = ['train', 'validation', 'test']
    
    for subset in subsets:
        for class_name in classes:
            dir_path = os.path.join(output_dir, subset, class_name)
            os.makedirs(dir_path, exist_ok=True)
    
    print(f"✅ Created output structure at: {output_dir}")
    print(f"\n📂 Structure:")
    print(f"   {os.path.basename(output_dir)}/")
    for subset in subsets:
        print(f"   ├── {subset}/")
        for i, class_name in enumerate(sorted(classes)):
            connector = "└──" if i == len(classes) - 1 else "├──"
            print(f"   │   {connector} {class_name}/")


# Create structure
classes = list(source_dataset.keys())
create_output_structure(OUTPUT_DIR, classes)

## 7. Copy & Augment Images

In [ ]:
def augment_and_save(image_path: str, output_dir: str, class_name: str,
                      base_name: str, aug_pipeline, n_augments: int) -> int:
    """
    Apply augmentation and save augmented images.
    Returns number of images saved.
    """
    saved = 0
    
    try:
        # Load image
        image = cv2.imread(image_path)
        if image is None:
            return 0
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        for i in range(n_augments):
            # Apply augmentation
            augmented = aug_pipeline(image=image)
            aug_image = augmented['image']
            
            # Convert back to BGR for saving
            aug_image_bgr = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)
            
            # Save augmented image
            aug_filename = f"{base_name}_aug{i+1}.jpg"
            output_path = os.path.join(output_dir, class_name, aug_filename)
            cv2.imwrite(output_path, aug_image_bgr, [cv2.IMWRITE_JPEG_QUALITY, 95])
            saved += 1
            
    except Exception as e:
        print(f"\n   ⚠️ Error augmenting {base_name}: {e}")
    
    return saved


def copy_images(splits: dict, output_dir: str, aug_pipeline, 
                augment_train: bool, aug_factor: int) -> dict:
    """
    Copy images to output directory and apply augmentation to training set.
    """
    stats = {
        'train': {'original': 0, 'augmented': 0, 'total': 0, 'by_class': {}},
        'validation': {'original': 0, 'total': 0, 'by_class': {}},
        'test': {'original': 0, 'total': 0, 'by_class': {}}
    }
    
    for subset_name, subset_data in splits.items():
        print(f"\n📁 Processing {subset_name} set...")
        subset_dir = os.path.join(output_dir, subset_name)
        
        for class_name, image_paths in subset_data.items():
            class_original = 0
            class_augmented = 0
            
            for img_path in tqdm(image_paths, desc=f"   {class_name}", leave=False):
                base_name = Path(img_path).stem
                
                # Copy original image
                dest_path = os.path.join(subset_dir, class_name, f"{base_name}.jpg")
                shutil.copy2(img_path, dest_path)
                class_original += 1
                
                # Apply augmentation only to training set
                if subset_name == 'train' and augment_train:
                    n_saved = augment_and_save(
                        img_path, subset_dir, class_name,
                        base_name, aug_pipeline, aug_factor
                    )
                    class_augmented += n_saved
            
            # Update stats
            stats[subset_name]['original'] += class_original
            stats[subset_name]['by_class'][class_name] = {
                'original': class_original,
                'augmented': class_augmented if subset_name == 'train' else 0,
                'total': class_original + (class_augmented if subset_name == 'train' else 0)
            }
            
            if subset_name == 'train':
                stats['train']['augmented'] += class_augmented
            
            print(f"   ✅ {class_name}: {class_original} original" + 
                  (f" + {class_augmented} augmented" if subset_name == 'train' and augment_train else ""))
        
        # Calculate totals
        stats[subset_name]['total'] = stats[subset_name]['original']
        if subset_name == 'train':
            stats['train']['total'] += stats['train']['augmented']
    
    return stats


# Process dataset
print("🚀 STARTING DATASET PROCESSING")
print("=" * 50)

processing_stats = copy_images(
    dataset_splits,
    OUTPUT_DIR,
    augmentation_pipeline,
    AUGMENT_TRAINING,
    AUGMENTATION_FACTOR
)

print("\n" + "=" * 50)
print("✅ PROCESSING COMPLETED!")

## 8. Generate Final Report

In [ ]:
def generate_report(stats: dict, output_dir: str) -> dict:
    """
    Generate comprehensive final report.
    """
    report = {
        'phase': 'Phase 4: Data Augmentation & Dataset Splitting',
        'timestamp': datetime.now().isoformat(),
        'configuration': {
            'source_dir': SOURCE_DIR,
            'output_dir': OUTPUT_DIR,
            'split_ratios': {
                'train': TRAIN_RATIO,
                'validation': VAL_RATIO,
                'test': TEST_RATIO
            },
            'augmentation': {
                'enabled': AUGMENT_TRAINING,
                'factor': AUGMENTATION_FACTOR,
                'target_size': TARGET_SIZE
            },
            'random_seed': RANDOM_SEED
        },
        'statistics': stats
    }
    
    # Save report
    report_path = os.path.join(output_dir, 'dataset_report.json')
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)
    
    return report


# Generate report
final_report = generate_report(processing_stats, OUTPUT_DIR)

# Display summary
print("\n" + "=" * 60)
print("📋 FINAL REPORT: Phase 4 Complete")
print("=" * 60)

print(f"\n📅 Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n📊 DATASET STATISTICS:")
print("-" * 40)

# Training set
print(f"\n🏋️ TRAINING SET:")
print(f"   Original images: {processing_stats['train']['original']}")
print(f"   Augmented images: {processing_stats['train']['augmented']}")
print(f"   Total: {processing_stats['train']['total']}")
for class_name, class_stats in sorted(processing_stats['train']['by_class'].items()):
    print(f"   └─ {class_name}: {class_stats['total']} ({class_stats['original']} + {class_stats['augmented']} aug)")

# Validation set
print(f"\n📊 VALIDATION SET:")
print(f"   Total: {processing_stats['validation']['original']}")
for class_name, class_stats in sorted(processing_stats['validation']['by_class'].items()):
    print(f"   └─ {class_name}: {class_stats['original']}")

# Test set
print(f"\n🧪 TEST SET:")
print(f"   Total: {processing_stats['test']['original']}")
for class_name, class_stats in sorted(processing_stats['test']['by_class'].items()):
    print(f"   └─ {class_name}: {class_stats['original']}")

# Grand total
grand_total = (processing_stats['train']['total'] + 
               processing_stats['validation']['original'] + 
               processing_stats['test']['original'])
print(f"\n📁 GRAND TOTAL: {grand_total} images")

print(f"\n💾 Report saved to: {OUTPUT_DIR}/dataset_report.json")

## 9. Verify Output Dataset

In [ ]:
def verify_dataset(output_dir: str) -> None:
    """
    Verify the final dataset structure and count images.
    """
    print("\n🔍 VERIFICATION")
    print("=" * 50)
    
    if not os.path.exists(output_dir):
        print("❌ Output directory does not exist!")
        return
    
    subsets = ['train', 'validation', 'test']
    
    for subset in subsets:
        subset_path = os.path.join(output_dir, subset)
        if not os.path.exists(subset_path):
            print(f"❌ Missing: {subset}/")
            continue
        
        print(f"\n📂 {subset}/")
        total = 0
        for class_name in sorted(os.listdir(subset_path)):
            class_path = os.path.join(subset_path, class_name)
            if os.path.isdir(class_path):
                count = len([f for f in os.listdir(class_path) 
                            if f.endswith(('.jpg', '.jpeg', '.png'))])
                total += count
                print(f"   ├── {class_name}/: {count} images")
        print(f"   └── Total: {total} images")
    
    print("\n✅ Verification complete!")


# Verify
verify_dataset(OUTPUT_DIR)

## 10. Preview Augmented Samples

In [ ]:
import matplotlib.pyplot as plt

def preview_augmented_samples(output_dir: str, samples_per_class: int = 4) -> None:
    """
    Display original and augmented samples from training set.
    """
    train_dir = os.path.join(output_dir, 'train')
    classes = sorted([d for d in os.listdir(train_dir) 
                      if os.path.isdir(os.path.join(train_dir, d))])
    
    if not classes:
        print("No classes found!")
        return
    
    fig, axes = plt.subplots(len(classes), samples_per_class, 
                             figsize=(14, 3.5 * len(classes)))
    
    if len(classes) == 1:
        axes = [axes]
    
    for i, class_name in enumerate(classes):
        class_path = os.path.join(train_dir, class_name)
        
        # Get mix of original and augmented
        all_images = [f for f in os.listdir(class_path) if f.endswith('.jpg')]
        originals = [f for f in all_images if '_aug' not in f][:2]
        augmented = [f for f in all_images if '_aug' in f][:2]
        samples = originals + augmented
        
        for j, img_name in enumerate(samples[:samples_per_class]):
            img_path = os.path.join(class_path, img_name)
            img = Image.open(img_path)
            
            ax = axes[i][j] if len(classes) > 1 else axes[j]
            ax.imshow(img)
            
            label_type = "AUG" if "_aug" in img_name else "ORIG"
            ax.set_title(f"{class_name}\n[{label_type}]", fontsize=9)
            ax.axis('off')
        
        # Fill empty slots
        for j in range(len(samples), samples_per_class):
            ax = axes[i][j] if len(classes) > 1 else axes[j]
            ax.axis('off')
    
    plt.tight_layout()
    plt.suptitle("Training Set Samples (Original vs Augmented)", fontsize=14, y=1.02)
    plt.show()


# Preview
preview_augmented_samples(OUTPUT_DIR)

## 11. Class Distribution Visualization

In [ ]:
def visualize_distribution(stats: dict) -> None:
    """
    Visualize class distribution across train/val/test sets.
    """
    classes = list(stats['train']['by_class'].keys())
    
    train_counts = [stats['train']['by_class'][c]['total'] for c in classes]
    val_counts = [stats['validation']['by_class'][c]['total'] for c in classes]
    test_counts = [stats['test']['by_class'][c]['total'] for c in classes]
    
    x = np.arange(len(classes))
    width = 0.25
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart
    bars1 = ax1.bar(x - width, train_counts, width, label='Train', color='#4CAF50')
    bars2 = ax1.bar(x, val_counts, width, label='Validation', color='#2196F3')
    bars3 = ax1.bar(x + width, test_counts, width, label='Test', color='#FF9800')
    
    ax1.set_xlabel('Disease Class')
    ax1.set_ylabel('Number of Images')
    ax1.set_title('Class Distribution Across Subsets')
    ax1.set_xticks(x)
    ax1.set_xticklabels(classes, rotation=15)
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            ax1.annotate(f'{int(height)}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3), textcoords="offset points",
                        ha='center', va='bottom', fontsize=8)
    
    # Pie chart for overall split
    sizes = [
        stats['train']['total'],
        stats['validation']['original'],
        stats['test']['original']
    ]
    labels = ['Train', 'Validation', 'Test']
    colors = ['#4CAF50', '#2196F3', '#FF9800']
    explode = (0.05, 0, 0)
    
    ax2.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
            shadow=True, startangle=90)
    ax2.set_title('Overall Dataset Split')
    
    plt.tight_layout()
    plt.show()


# Visualize
visualize_distribution(processing_stats)

---

## ✅ Phase 4 Complete!

The augmented and split dataset is ready at:
```
/content/drive/MyDrive/DrukFarm/data/final_potato_dataset/
├── train/
│   ├── Early_Blight/   (original + 3x augmented)
│   ├── Late_Blight/    (original + 3x augmented)
│   └── Healthy/        (original + 3x augmented)
├── validation/
│   ├── Early_Blight/   (original only)
│   ├── Late_Blight/    (original only)
│   └── Healthy/        (original only)
└── test/
    ├── Early_Blight/   (original only)
    ├── Late_Blight/    (original only)
    └── Healthy/        (original only)
```

**Key Achievements:**
- ✅ Stratified 70/20/10 split
- ✅ Class balance maintained
- ✅ Training set augmented 3x
- ✅ No data leakage between sets
- ✅ Reproducible with seed=42

**Next Steps:**
- Phase 5: Model Training